# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Viedma,-40.8135,-62.9967,68.50,50,100,20.33,AR,2023-12-11 02:51:12
1,1,Kingston,17.9970,-76.7936,83.35,76,40,20.71,JM,2023-12-11 02:47:10
2,2,Port-Aux-Francais,-49.3500,70.2167,39.78,77,78,20.98,TF,2023-12-11 02:49:29
3,3,Kalmunai,7.4167,81.8167,80.20,82,92,5.21,LK,2023-12-11 02:51:12
4,4,Grytviken,-54.2811,-36.5092,41.88,95,100,14.36,GS,2023-12-11 02:49:31


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", "Lat", geo = True, color = "City", size = "Humidity", tiles = "OSM")

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_narrow=city_data_df[(city_data_df["Max Temp"] < 80) &
                             (city_data_df["Max Temp"] > 60) &
                             (city_data_df["Wind Speed"] < 10) &
                             (city_data_df["Cloudiness"] < 15) &
                             (city_data_df["Humidity"] < 40)]

city_data_narrow=city_data_narrow.dropna() # drop any rows will null values

city_data_narrow.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,75,Margaret River,-33.9500,115.0667,77.47,31,0,5.44,AU,2023-12-11 02:49:31
101,101,New Norfolk,-42.7826,147.0587,70.92,25,11,3.27,AU,2023-12-11 02:49:32
264,264,Ubauro,28.1652,69.7289,64.45,20,0,4.83,PK,2023-12-11 02:51:37
287,287,Umarkot,25.3616,69.7362,66.74,36,0,4.52,PK,2023-12-11 02:51:39
376,376,Kachia,9.8667,7.9500,66.18,25,0,8.55,NG,2023-12-11 02:51:46


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Viedma,AR,-40.8135,-62.9967,50,
1,Kingston,JM,17.9970,-76.7936,76,
2,Port-Aux-Francais,TF,-49.3500,70.2167,77,
3,Kalmunai,LK,7.4167,81.8167,82,
4,Grytviken,GS,-54.2811,-36.5092,95,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000 
api_key = "e2bf48dad67742859cf8f0456a673314" 
params = {"categories": "accommodation.hotel",
          "apiKey": api_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add filter parameter with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        
        # Convert the API response to JSON format
        name_address = response.json()
        
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
            
        # Log the search results
        print(f"{hotel_df.loc[index, "City"]} - nearest hotel: {hotel_df.loc[index, "Hotel Name"]}")
        
    else:
        # Print an error message if the request was not successful
        print(f"Error: {response.status_code}")

# Display sample data
hotel_df


Starting hotel search
Viedma - nearest hotel: Hostel EL NOGAL
Kingston - nearest hotel: The Kingston Hilton Hotel
Port-Aux-Francais - nearest hotel: Keravel
Kalmunai - nearest hotel: Sakkaryya Hotel and Restaurant
Grytviken - nearest hotel: No hotel found
Illizi - nearest hotel: فندق الطيب العربي مولاي
Udachny - nearest hotel: Вилюй
Laguna - nearest hotel: TownePlace Suites Sacramento Elk Grove
San Patricio - nearest hotel: No hotel found
Adamstown - nearest hotel: No hotel found
Puerto Ayora - nearest hotel: Deja Vú
Tiksi - nearest hotel: Арктика
Itamaraca - nearest hotel: Pousada Vento Leste
Tual - nearest hotel: SURYA INN
Sao Vicente - nearest hotel: Ibis Budget Santos Gonzaga
Badger - nearest hotel: Candlewood Suites Fairbanks
La Ronge - nearest hotel: The Harbour Inn
Brookings - nearest hotel: Country Inn & Suites
Kodiak - nearest hotel: Kodiak MWR Guest House
Portel - nearest hotel: No hotel found
Bethel - nearest hotel: Hotel Zero Degrees
Dakhla - nearest hotel: Bab Al Bahar Hot

Hermiston - nearest hotel: Holiday Inn Express & Suites;Holiday Inn Express & Suites Hermiston Downtown
Mindelo - nearest hotel: Arla Residencial
Rongelap - nearest hotel: No hotel found
Price - nearest hotel: Holiday Inn Express Price
Xilin Hot - nearest hotel: Xilinguole Shitai International Hotel
Tolanaro - nearest hotel: Kaleta Hotel
Griffith - nearest hotel: Residence Inn by Marriott Merrillville
Gualeguay - nearest hotel: No hotel found
Moussoro - nearest hotel: No hotel found
Merizo Village - nearest hotel: No hotel found
Fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
Fale Old Settlement - nearest hotel: No hotel found
Tra Mhor - nearest hotel: Majestic Hotel
Rabo De Peixe - nearest hotel: Hotel Marina Atlântico
St. John'S - nearest hotel: DoubleTree
Guozhen - nearest hotel: No hotel found
Dokkum - nearest hotel: Hotel De Posthoorn
Akdepe - nearest hotel: No hotel found
Chardara - nearest hotel: Частная гост.
Klyuchi - nearest hotel: No hotel found
Nova Sintra - n

Fairmont - nearest hotel: Holiday Inn Express
Ulsteinvik - nearest hotel: Quality Hotel Ulstein
Tazovsky - nearest hotel: Тазовчанка
Kayenta - nearest hotel: Hampton Inn Kayenta
Kachia - nearest hotel: No hotel found
Hadibu - nearest hotel: No hotel found
Chernyshevskiy - nearest hotel: No hotel found
Major Isidoro - nearest hotel: Hotel Terra do Leite
Mana - nearest hotel: Le Samana
Saipan - nearest hotel: Hyatt Regency Saipan
Posse - nearest hotel: Palmeiras Hotel
Ko Lanta - nearest hotel: Mango House
Nizhnevartovsk - nearest hotel: Надежда
Balqash - nearest hotel: Квартиры по суточно
Puerto Aysen - nearest hotel: Hotel Aysén Patagonia
Ribeira Grande - nearest hotel: Azoris Faial Garden - Resort Hotel
La'Ie - nearest hotel: Hale Koolau
Umba - nearest hotel: Кедр Грасс Белое море
Viana Do Castelo - nearest hotel: Hotel Laranjeira
Padrauna - nearest hotel: No hotel found
Qarazhal - nearest hotel: No hotel found
Narra - nearest hotel: Gulane Travellers' Home
Selfoss - nearest hotel: Hot

Labuan - nearest hotel: OYO 969 Hotel Pantai View
Baymak - nearest hotel: No hotel found
Rusape - nearest hotel: No hotel found
South Hedland - nearest hotel: Port Haven
Barguzin - nearest hotel: No hotel found
Mandan - nearest hotel: Fairfield Inn & Suites Bismarck North
Aswan - nearest hotel: Mövenpick Resort Aswan
Labrador City - nearest hotel: Northern Inn & Suites
Horsham - nearest hotel: Wimblehurst Hotel
Xai-Xai - nearest hotel: No hotel found
Noumea - nearest hotel: Hilton Noumea La Promenade Residences
Tianshui - nearest hotel: 天水金龙大酒店
Honiara - nearest hotel: Solomon Kitano Mendana Hotel
Turabah - nearest hotel: No hotel found
Nelson Bay - nearest hotel: Mantra chip
San Mateo Del Mar - nearest hotel: No hotel found
Portsmouth - nearest hotel: The Queens Hotel
Toliara - nearest hotel: Moringa Zomatel
Ubari - nearest hotel: فندق اوباري
Al Kharijah - nearest hotel: Sol-Y-Mar Pioneers
Cruz Grande - nearest hotel: No hotel found
Port Said - nearest hotel: Helton Hotel
Ambasa - nea

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Viedma,AR,-40.8135,-62.9967,50,Hostel EL NOGAL
1,Kingston,JM,17.9970,-76.7936,76,The Kingston Hilton Hotel
2,Port-Aux-Francais,TF,-49.3500,70.2167,77,Keravel
3,Kalmunai,LK,7.4167,81.8167,82,Sakkaryya Hotel and Restaurant
4,Grytviken,GS,-54.2811,-36.5092,95,No hotel found
...,...,...,...,...,...,...
593,Tallkayf,IQ,36.4894,43.1192,77,No hotel found
594,Madang,PG,-5.2246,145.7966,57,Madang Kekri Lodge
595,Mahina,PF,-17.5065,-149.4890,62,Hôtel Sarah Nui
596,Doembang Nangbuat,TH,14.8333,100.1000,60,Tharnrarin Villa


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display
hotel_df.hvplot.points(
    geo=True,
    x="Lng",
    y="Lat",
    size=10,
    hover_cols=["City", "Hotel Name", "Country"],
    title="Nearest Hotels",
    color="Hotel Name",
    cmap="Category20",  
    tiles="OSM"
).opts(
    tools=["hover"],
)



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,City,Country)